# Analysis with currents from CMEMS

Here we will use currents from [CMEMS](https://marine.copernicus.eu/) to illustrate the use of an advection constrain.      
The data access is now provided through the Copernicus [Marine Data Store](https://marine.copernicus.eu/news/introducing-new-copernicus-marine-data-store) (the `motu` client has been discontinued since April 2024).

In this application we use a subset of the CMEMS data for the year 2018.
## Load modules

In [ ]:
using NCDatasets
using DIVAnd
using Makie, CairoMakie, GeoMakie
using Dates
using Statistics
include("../config.jl")

## Set domain
The domain is identifical to the full example notebook.

In [ ]:
dx, dy = 0.125, 0.125
lonr = 11.5:dx:20
latr = 39:dy:46
depthr = [0.0, 10.0, 20.0];

## Load data
### Observations
We work with salinity measurements from SeaDataNet.

In [ ]:
download_check(largeadriaticfile, largeadriaticfileURL)
varname = "Salinity"
@time obsval, obslon, obslat, obsdepth, obstime, obsid = ODVspreadsheet.load(
    Float64,
    [largeadriaticfile],
    ["Water body salinity"];
    nametype = :localname,
);

checkobs((obslon, obslat, obsdepth, obstime), obsval, obsid)

Perform a simple range check.

In [ ]:
sel = (obsval .<= 40) .& (obsval .>= 25);

obsval = obsval[sel]
obslon = obslon[sel]
obslat = obslat[sel]
obsdepth = obsdepth[sel]
obstime = obstime[sel]
obsid = obsid[sel];

### Velocity
For simplicity we download an already prepared netCDF file storing the velocity.

In [ ]:
download_check(velfilename, velfilenameURL)

yearlist = [1900:2017];
monthlist = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]];
monthlist = [[1, 2, 3]];
TSvelocity = DIVAnd.TimeSelectorYearListMonthList(yearlist, monthlist);

varnames = ("vozocrtx", "vomecrty")
outvarnames = ("u", "v")
outfilevel = joinpath(outputdir, "average-currents.nc")

if isfile(outfilevel)
    rm(outfilevel)
end

We use the function `DIVAnd.average_files` to average the gridded variables from the a set of netCDF files over the selected time period. The first argument of `DIVAnd.average_files` can also be a list of filenames.

In [ ]:
DIVAnd.average_files([velfilename], varnames, TSvelocity, outfilevel, outvarnames)

### Load the bathymetry

In [ ]:
bathname = gebco08file
download_check(gebco08file, gebco08fileURL)
bx, by, b = load_bath(bathname, true, lonr, latr);
_, _, mask = load_mask(bathname, true, lonr, latr, depthr)

### Prepare the mask

In [ ]:
mask = falses(size(b, 1), size(b, 2), length(depthr))
for k = 1:length(depthr)
    for j = 1:size(b, 2)
        for i = 1:size(b, 1)
            mask[i, j, k] = b[i, j] >= depthr[k]
        end
    end
end
@show size(mask)

grid_bx = [i for i in bx, j in by];
grid_by = [j for i in bx, j in by];

mask_edit = copy(mask);
sel_mask1 = (grid_by .<= 42.6) .& (grid_bx .<= 14.0);
sel_mask2 = (grid_by .<= 41.2) .& (grid_bx .<= 16.2);
mask_edit = mask_edit .* .!sel_mask1 .* .!sel_mask2;
@show size(mask_edit)

### Read the winter average

In [ ]:
ds = Dataset(outfilevel);
u = nomissing(ds["u"][:, :, 1, 1], NaN);
v = nomissing(ds["v"][:, :, 1, 1], NaN);
uvlon = nomissing(ds["lon"][:]);
uvlat = nomissing(ds["lat"][:])
close(ds)

### Make plot

In [ ]:
fig = Figure();
r = 2;
ga = GeoAxis(
    fig[1, 1];
    dest = "+proj=merc",
    title = "Velocity field",
    xticks = 8:4:20,
    yticks = 38:2:46,
)
heatmap!(ga, bx, by, mask_edit[:, :, 1], colormap = Reverse(:binary))
arrows!(
    ga,
    uvlon[1:r:end],
    uvlat[1:r:end],
    u[1:r:end, 1:r:end],
    v[1:r:end, 1:r:end],
    arrowsize = 4,
    lengthscale = 4,
)
xlims!(lonr[1], lonr[end])
ylims!(latr[1], latr[end])
# Reference vector
arrows!(ga, [18.0], [44.5], [0.25], [0.0], color = :white, arrowsize = 4, lengthscale = 4)
text!(ga, 18, 44.6, text = "0.25 m/s", color = :white)
fig

## Analysis
### Set parameter values

In [ ]:
#@time rdiag=1.0./DIVAnd.weight_RtimesOne((obslon,obslat),(0.03,0.03));
#@show maximum(rdiag),mean(rdiag)

sz = (length(lonr), length(latr), length(depthr));
lenx = fill(100_000.0, sz)   # 100 km
leny = fill(100_000.0, sz)   # 100 km
lenz = fill(25.0, sz);      # 25 m
len = (lenx, leny, lenz);
epsilon2 = 0.1;
#epsilon2 = epsilon2 * rdiag;

outputfile =
    joinpath(outputdir, "Water_body_$(varname)_Adriatic_advection_constraint.4Danl.nc")

### Define function to create plots

In [ ]:
figdir = "../figures/Adriatic/"
isdir(figdir) ? @debug("Figure directory already exists") : mkpath(figdir)

In [ ]:
function plotres(timeindex, sel, fit, erri)
    tmp = copy(fit)
    tmp[erri.>0.5] .= NaN

    for i = 1:length(depthr)
        fig = Figure()

        # select the data near the surface
        selsurface = (obsdepth .< 5)
        vmin, vmax = quantile(obsval[selsurface], (0.01, 0.99))

        # plot the data
        ga = GeoAxis(fig[1, 1]; dest = "+proj=merc", title = "Salinity at $(depthr[i]) m")
        heatmap!(ga, lonr, latr, mask_edit[:, :, 1], colormap = Reverse(:binary))
        hm = heatmap!(
            ga,
            lonr,
            latr,
            tmp[:, :, i],
            colorrange = [33.0, 40.0],
            colormap = :haline,
        )

        xlims!(ga, (lonr[1], lonr[end]))
        ylims!(ga, (latr[1], latr[end]))
        Colorbar(fig[2, 1], hm, vertical = false)

        figname =
            varname *
            lpad(i, 2, '0') *
            lpad(timeindex, 3, '0') *
            "_advection_constraint.png"
        save(joinpath(figdir, figname), fig)
        display(fig)
    end
end

### Run analysis using velocity from file
We use the function `DIVAnd.velocityfile`, which allows one to specify the velocity field used for the advection constrain.

In [ ]:
?DIVAnd.velocityfile

In [ ]:
TS = TSvelocity
isfile(outputfile) ? rm(outputfile) : @debug("Output file doesn't exist")

@time dbinfo = diva3d(
    (lonr, latr, depthr, TS),
    (obslon, obslat, obsdepth, obstime),
    obsval,
    len,
    epsilon2 / 50,
    outputfile,
    varname,
    bathname = bathname,
    plotres = plotres,
    mask = mask_edit,
    fitcorrlen = false,
    surfextend = true,
    velocity = DIVAnd.velocityfile(outfilevel, ("u", "v"), TSvelocity, 24 * 60 * 60 * 10),
);